# Plot N cross-section with matplotlib

In [1]:
%matplotlib widget

In [2]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
from JulesD3D.processNetCDF import addDepth
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
import numpy as np

In [3]:
# ncfilename = '/Volumes/Seagate Expansion Drive/ThesisResults/trim-36km_200m_W60ChannelRun01MoreOutput_compressedmore.nc'
ncfilename = '/Users/julesblom/ThesisResults/trim-36km_200m_W60ChannelRun01_compressed.nc'
trim = xr.open_dataset(ncfilename) # Lazyload DataSet with xarray, unload from memory with .close()

In [4]:
# only if the NetCDF has already been processed
if "depth_center" not in trim:
    print("Need to processNetCDF first")
    trim = addDepth(trim)

depth_center = trim['depth_center']
depth_center

Need to processNetCDF first


/Users/julesblom/Library/Caches/pypoetry/virtualenvs/delft3d-python-j5EFwY34-py3.7/lib/python3.7/site-packages/xarray/core/computation.py:1215: FutureWarning: This DataArray contains multi-dimensional coordinates. In the future, these coordinates will be transposed as well unless you specify transpose_coords=False.
  return result.transpose(*[d for d in all_dims if d in result.dims])


<xarray.DataArray 'depth_center' (time: 13, M: 132, N: 182, SIG_LYR: 80)>
array([[[[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ]],

        [[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [ -25.101604,  -71.17948 , -111.384   , ..., -698.64966 ,
          -698.8944  , -699.1041  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ]],

        [[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [ -25.101604,  -71.17948 , -111.384   , ..., -698.64966 ,
          -698.8944  , -699.1041  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ]],

        ...,

        [[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [ -25.101604,  -71.17948 , -111.384   , ..., -698.64966 ,
          -698.8944  , -699.1041  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ]],

        [[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [ -25.101604,  -71.17948 , -111.384   , ..., -698.64966 ,
          -698.8944  , -699.1041  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ],
         [ -25.126667,  -71.25055 , -111.49521 , ..., -699.3472  ,
          -699.5922  , -699.8022  ]],

        [[   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         ...,
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ],
         [   0.      ,    0.      ,    0.      , ...,    0.      ,
             0.      ,    0.      ]]],


       [[[   0.      ,    0.      ,    0.      , ...,    0.      ,
   

In [5]:
selected_time = 2
selected_M = int(trim.M.size/2)

density_m_at_time = trim.RHO.isel(time=selected_time, M=selected_M) 
z_section_center = depth_center.isel(time=selected_time, M=selected_M) # density_m_31.depth_center.T #trim.depth_center.isel(time=this_time, M=this_M).T # why do i need to transpose this?

z_section_intf = trim.depth.isel(time=selected_time, M=selected_M) # why do i need to transpose this?

n_section = z_section_intf.YZ

_, mesh_N = np.meshgrid(trim.SIG_INTF.values, n_section)

### Sanity check of dimensions

In [6]:
# is there a way to quickly print a formatted table in jupyter without pandas?
print('quad_NSigma\t', mesh_N.shape)
print('n_section\t', n_section.shape)
print('z_section_center', z_section_center.shape)
print('z_section_intf\t', z_section_intf.shape)
print('density_m_at_time', density_m_at_time.T.shape)

quad_NSigma	 (182, 81)
n_section	 (182,)
z_section_center (182, 80)
z_section_intf	 (182, 81)
density_m_at_time (182, 80)


In [7]:
sigma_percentages = np.diff(trim.SIG_LYR.values)*-100

In [8]:
bottom = z_section_intf.isel(SIG_INTF=-1)

In [9]:
legend_hack_intf = z_section_intf.isel(SIG_INTF=-1)
legend_hack_center = z_section_center.isel(SIG_LYR=-1)

In [10]:
z_section_intf.to_pandas().to_csv('/Users/julesblom/Downloads/interfaces.csv')

In [11]:
plt.close("all")

In [32]:
not_all_centers = z_section_center.isel(SIG_LYR=slice(0,40))

## Plot Sigma interfaces and center with length

In [33]:
# zscale = 20
fig, ax = plt.subplots(nrows=1, figsize=(10,4.5)) # scale to actual depth lenght: (20 ,4) for  factor 10
plt.suptitle('Vertical grid-section')
plt.title("Channel slope 1.5° - Basin slope 0.2°", fontsize=11)

ax.plot(n_section, z_section_intf, c='k', linewidth=0.55, marker='+', markersize=0.1)
ax.plot(n_section, not_all_centers, c='steelblue', linewidth=0.15, marker='+', markersize=0.1)

ax.xaxis.set_minor_locator(MultipleLocator(1000))
plt.xlim(0., 36200)

# hack to get only names in legend for lines
interfaces, = ax.plot(n_section, legend_hack_intf, c='k', linewidth=0.8, marker='+', markersize=0.)
centers, = ax.plot(n_section, legend_hack_center, c='steelblue', linewidth=0.8, marker='+', markersize=0.1)

# mark one line just because
# ax.plot(n_section, z_section_intf[:,42], c='hotpink', marker='o', markersize=1.5,)

# inset axes
axins = ax.inset_axes([0.5, 0.5, 0.47, 0.47])
axins.plot(n_section, z_section_intf, c='k',  linewidth=0.55, marker='+', markersize=0.1)
plt.setp(axins.spines.values(), color='tomato')
plt.setp([axins.get_xticklines(), ax.get_yticklines()], color='tomato')
plt.setp([axins.get_yticklines(), ax.get_yticklines()], color='tomato')


# sub region of the original image
x1, x2 = 13000, 17000
y1, y2 = -635, -600
axins.set_xlim(x1, x2)
axins.set_ylim(y1, y2)
axins.set_xticklabels('')
axins.set_yticklabels('')

ax.indicate_inset_zoom(axins, edgecolor='tomato', alpha=0.5)

# ax.scatter(n_section, depth_m_31[:,10], s=1.5)
ax.set_xlabel('Length $n$ [m]', fontsize=13.5)
ax.set_ylabel('Depth [m]', fontsize=13.5)
ax.legend((interfaces, centers), ('$\sigma$ Interfaces', '$\sigma$ Centers'), fontsize="x-large", loc="lower left", markerscale=50)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
# help(ax.indicate_inset_zoom)
# plt.close('all')

## Plot Sigma interfaces and center with grid index

In [15]:
fig_N, ax_N = plt.subplots(nrows=1, figsize=(9,6))
ax_N.set_title('Sigma layers vertical section - N number')

ax_N.plot(z_section_intf, c='k', linewidth=0.25, marker='+', markersize=0.1)
ax_N.plot(z_section_center, c='g', linewidth=0.05, marker='+', markersize=0.1)
ax_N.set_xlabel('N')
ax_N.set_ylabel('Depth [m]')
fig_N.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Plot density N cross section

In [16]:
print(mesh_N[:,:].shape)
print(z_section_intf[:,:].shape)
print(density_m_at_time.T.shape)

(182, 81)
(182, 81)
(182, 80)


In [17]:
fig_vert, ax_vert2 = plt.subplots(nrows=1, figsize=(9,6))

ax_vert2.set_title('Vertical Dection Density')

# or z_section.YZ both work
mesh = ax_vert2.pcolormesh(mesh_N, z_section_intf, density_m_at_time.T,
                          vmin=1025, vmax=1030) #, edgecolors='w', linewidths=0.1)
# ax_vert2.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10, alpha=0.5) #, marker='+', markersize=0.1)
# ax_vert2.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.08, alpha=0.5)

# ax_vert2.plot(n_section, z_section_intf[:,42], c='hotpink', marker='o', markersize=1.5,)


cbar = fig_vert.colorbar(mesh, ax=ax_vert2)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Density [kg/m3]',rotation=90)
ax_vert2.set_xlabel('Length $n$ [m]')
ax_vert2.set_ylabel('Depth [m]')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

In [26]:
fig_vert, ax_vert1 = plt.subplots(nrows=1, figsize=(9,6))

ax_vert1.set_title('Vertical Section Density')
ax_vert1.set_xlim(250, 2000)
ax_vert1.set_ylim(-400, -200)

# or z_section.YZ both work
mesh = ax_vert1.pcolormesh(mesh_N, z_section_intf, density_m_at_time.T,
                          vmin=1025, vmax=1035) #, edgecolors='w', linewidths=0.1)
# ax_vert1.plot(n_section, z_section_center, c='lightsteelblue', linewidth=0.10, alpha=0.5) #, marker='+', markersize=0.1)
ax_vert1.plot(n_section, z_section_intf[:,:], c='bisque', linewidth=0.1, alpha=1, marker='+', markersize=0.05)
ax_vert1.plot(n_section, z_section_intf[:,-53], c='white', marker='o', markersize=1.5,)
# ax_vert1.plot(n_section, z_section_intf[:,-53:None], c='white', linewidth=0.06,)
sigma_text = ax_vert1.text(1000, -275, "$\sigma$-layer = 27", color="white", fontsize=13, rotation=-7)
sigma_text.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='black')])


cbar = fig_vert.colorbar(mesh, ax=ax_vert1)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('Density [$kg/m^3$]', rotation=90, fontsize=13)
ax_vert1.set_xlabel('Length $n$ [m]', fontsize=14)
ax_vert1.set_ylabel('Depth [m]', fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Depth [m]')

In [ ]:
plt.close("all")